In [1]:
# Analytics Expression Language Example

import xarray as xr
from datetime import datetime
from datacube.api import API
from datacube.ndexpr import NDexpr

In [2]:
# Instantiating API and NDexpr

g = API()
nd = NDexpr()

In [3]:
# construct data request parameters for band_30 and band_40
data_request_descriptor = {
    'platform': 'LANDSAT_5',
    'product': 'nbar',
    'variables': ('band_3', 'band_4'),
    'dimensions': {
        'x': {
            'range': (150, 150.256)
        },
        'y': {
            'range': (-34.0, -33.744)
        },
        'time': {
            'range': (datetime(1990, 1, 1), datetime(1990, 12, 31))
        }
    }
}

# Retrieving data from API
d1 = g.get_data(data_request_descriptor)

# construct data request parameters for PQ
pq_request_descriptor = {
    'platform': 'LANDSAT_5',
    'product': 'pqa',
    'variables': ('pixelquality'),
    'dimensions': {
        'x': {
            'range': (150, 150.256)
        },
        'y': {
            'range': (-34.0, -33.744)
        },
        'time': {
            'range': (datetime(1990, 1, 1), datetime(1990, 12, 31))
        }
    }
}

# Retrieving data from API
d2 = g.get_data(pq_request_descriptor)

In [4]:
# The following 3 lines shouldn't be done like this
# Currently done like this for the sake of the example.

b30 = xr.DataArray(d1['arrays']['band_3'])
b40 = xr.DataArray(d1['arrays']['band_4'])
pq = nd.get_pqa_mask(d2['arrays']['pixelquality'].values)

In [5]:
# NDexpr demo begins here

# perform ndvi as expressed in this language.
ndvi = nd.evaluate('((b40 - b30) / (b40 + b30))')

# perform mask on ndvi as expressed in this language.
masked_ndvi = nd.evaluate('ndvi{pq}')

In [6]:
print(ndvi)

<xarray.DataArray (time: 12, y: 1264, x: 1094)>
array([[[-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        ..., 
        [ 0.72546012,  0.75293306,  0.72474747, ...,  0.60863874,
          0.74612292,  0.69976905],
        [ 0.72546012,  0.7819732 ,  0.74063058, ...,  0.62055838,
          0.74612292,  0.73892499],
        [ 0.76122673,  0.72474747,  0.76828479, ...,  0.65077233,
          0.75745527,  0.71428571]],

       [[-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        ..., 
    

In [7]:
print(masked_ndvi)

<xarray.DataArray (time: 12, y: 1264, x: 1094)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ..., 
        [ 0.72546012,  0.75293306,  0.72474747, ...,  0.60863874,
          0.74612292,  0.69976905],
        [ 0.72546012,  0.7819732 ,  0.74063058, ...,  0.62055838,
          0.74612292,  0.73892499],
        [ 0.76122673,  0.72474747,  0.76828479, ...,  0.65077233,
          0.75745527,  0.71428571]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ..., 
    

In [8]:
# currently dimensions are integer indices, later will be labels when
# Data Access API Interface has been finalised.

reduction_on_dim0 = nd.evaluate('median(masked_ndvi, 0)')
print(reduction_on_dim0)

<xarray.DataArray (y: 1264, x: 1094)>
array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ..., 
       [ 0.7421573 ,  0.76299446,  0.7630921 , ...,  0.65916716,
         0.68892367,  0.68132084],
       [ 0.7899478 ,  0.7877796 ,  0.7700596 , ...,  0.65662187,
         0.6819106 ,  0.66060538],
       [ 0.78449283,  0.79193582,  0.7850524 , ...,  0.65284442,
         0.68658489,  0.6886646 ]])
Coordinates:
  * y        (y) float64 -3.799e+06 -3.799e+06 -3.799e+06 -3.799e+06 ...
  * x        (x) float64 1.649e+06 1.649e+06 1.649e+06 1.649e+06 1.649e+06 ...


In [9]:
reduction_on_dim01 = nd.evaluate('median(masked_ndvi, 0, 1)')
print(reduction_on_dim01)

<xarray.DataArray (x: 1094)>
array([ 0.68826291,  0.68593686,  0.68199249, ...,  0.71585299,
        0.71710697,  0.71660741])
Coordinates:
  * x        (x) float64 1.649e+06 1.649e+06 1.649e+06 1.649e+06 1.649e+06 ...


In [10]:
reduction_on_dim012 = nd.evaluate('median(masked_ndvi, 0, 1, 2)')
print(reduction_on_dim012)

<xarray.DataArray ()>
array(0.707667731629393)
